In [1]:
import pandas as pd
import numpy as np

In [2]:
ohlcv = pd.read_csv('binance-btc-usdc.csv.gz', index_col='dt')

In [3]:
cdiff = ohlcv.close.pct_change().fillna(0)
dir = pd.Series(0, index=ohlcv.index)
dir[cdiff > .001] = 1
dir[cdiff < -.001] = 2
ohlcv['dir'] = dir.shift(-1)
ohlcv = ohlcv.iloc[:-1]
ohlcv.dir = ohlcv.dir.astype(int)

In [4]:
ohlcv_h = pd.concat((ohlcv, ohlcv.shift(1).add_prefix('p_'), ohlcv.shift(2).add_prefix('pp_')), axis=1)
ohlcv_h = ohlcv_h.iloc[2:]
ohlcv_h.dir = ohlcv_h.dir.astype(int)

In [5]:
train = ohlcv_h.loc[:'2020-02-01']
test = ohlcv_h.loc['2020-02-01':]

In [6]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier(learning_rate=0.005, n_estimators=200)
gbm.fit(train.drop(columns='dir'), train.dir)
pred = gbm.predict_proba(test.drop(columns='dir'))

from sklearn.metrics import roc_auc_score
roc_auc_score(test.dir, pred, multi_class='ovo', average='weighted')

0.5749037245205024

In [65]:
actions = pred.argmax(axis=1)
scales = 1+((test.close.shift(-1) - test.close)/test.close).fillna(0)

long = scales[actions==1].prod()
short = (1/scales[actions==2]).prod()

# long if 1, short if 2 cash if 0, no commissions
long*short

2.074305364926968